In [11]:
import pandas as pd
from scipy.stats import kurtosis
import matplotlib.pyplot
import seaborn as sb

In [12]:
df = pd.read_excel('./Grupo 01.xlsx')
#TODO: medidas de tendencia central
#TODO: medidas de variacion
#TODO: cuantitativas cosntruir un grafico
#TODO: tabla de frecuencia para las cualitativas
#TODO: grafico para cualitativas
#TODO: medidas de tendencia central para precio segun el parqueadero
#TODO: medias de tendencia central para mt2 segun si esta terminaod o no
#TODO: box-plot para las ultimas dos tareas

## Center's Measurement

In [42]:
variables = ['precio','mt2','estrato','alcobas','banos','administracion','avaluo']

var_media = [round(df[i].mean(),2) for i in variables]
var_mediana = [round(df[i].median(),2) for i in variables]
var_mode = [round(df[i].mode(),2) for i in variables]

df_variables = pd.DataFrame(var_media)
df_variables.columns = ['Media']
df_variables['Mediana'] = var_mediana
df_variables['Moda'] = var_mode

## Scattering's Measurement

In [57]:
var_std = [round(df[i].std(),2) for i in variables]
# var_std
var_min = [df[i].min() for i in variables]
# var_min
var_max = [df[i].max() for i in variables]
# var_max
var_q1 = [round(df[i].quantile(q=0.25),2) for i in variables]
# var_q1
var_q3 = [round(df[i].quantile(q=0.75),2) for i in variables]
# var_q3
var_var = [round(df[i].var(),2) for i in variables]
# var_var
var_asi = [round(df[i].skew(),2) for i in variables]
# var_asi
var_kur = [round(kurtosis(df[i], fisher=False),2) for i in variables]
# var_kur
var_range = [df[i].max() - df[i].min() for i in variables]
# var_range

[1675.0, 474.0, 4, 4, 4, 2.28, 1517.9910000000002]

### Cualitative variable types

In [ ]:
def get_acum (dataframe,option):
    """Funcion que llena un arreglo <storaged> con los valores acumulados
       de la variable que el pasemos como parametro en <values>
    """
    acum = 0
    storaged = []

    if option == 1:
       values = dataframe['Frecuencia_absoluta'].values

       for i in values:
            acum = acum + i
            storaged.append(acum)

       dataframe['Frec_absoluta_acumulada'] = storaged

    elif option == 2:
        values = dataframe['Frecuencia_relativa'].values

        for i in values:
            acum = acum + i
            storaged.append(acum)

        dataframe['Frec_relativa'] = storaged

def get_frec_ab (values):
    frec_ab = values.value_counts()
    frec_df = pd.DataFrame(frec_ab)
    frec_df.columns = ['Frecuencia_absoluta']
    return frec_df

def get_frec_rel (dataframe,source):
    dataframe['Frecuencia_relativa'] = round(dataframe['Frecuencia_absoluta'] * 100 / len(source),3)


In [ ]:
var_ubicacion = df['ubicacion']

'''Frecuencia absoluta'''

frec_ubicacion = get_frec_ab(var_ubicacion)

'''Frecuencia relativa'''

get_frec_rel(frec_ubicacion,var_ubicacion)

'''Frecuencia absoluta acumulada'''
get_acum(frec_ubicacion,1)

'''Frecuencia relativa acumulada'''

get_acum(frec_ubicacion,2)

In [ ]:
var_balcon = df['balcon']


'''Frecuencia absoluta'''

frec_balcon = get_frec_ab(var_balcon)

'''Frecuencia relativa'''

get_frec_rel(frec_balcon,var_balcon)

'''Frecuencia absoluta acumulada'''
get_acum(frec_balcon,1)

'''Frecuencia relativa acumulada'''
get_acum(frec_balcon,2)


In [ ]:
var_parqueadero = df['parqueadero']

'''Frecuencia absoluta'''

frec_parqueadero = get_frec_ab(var_parqueadero)

'''Frecuencia relativa'''

get_frec_rel(frec_parqueadero,var_parqueadero)

'''Frecuencia absoluta acumulada'''
get_acum(frec_parqueadero,1)

'''Frecuencia relativa acumulada'''
get_acum(frec_parqueadero,2)

In [ ]:
var_terminado = df['terminado']

'''Frecuencia absoluta'''

frec_terminado = get_frec_ab(var_terminado)

'''Frecuencia relativa'''

get_frec_rel(frec_terminado,var_terminado)

'''Frecuencia absoluta acumulada'''
get_acum(frec_terminado,1)

'''Frecuencia relativa acumulada'''
get_acum(frec_terminado,2)
